# 可视化

使用wandb进行mask可视化, 需要以下：
1. image
2. pred_mask
3. gt_mask

In [1]:
import wandb

masked_image = wandb.Image(
    image,
    mask={
        "prediction": {"mask_data": pred_mask}
        "groud_truth": {"mask_data": gt_mask}
    }
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1073629662.py, line 6)

从数据集取出来的image和label都是在0，1之间，且image已经被transposed了，所以需要对其进行处理

In [ ]:
import torch
def convert_image_mask(image, mask):
    """
    image 是一个(1,3,H,W)，值位于0到1之间的张量，类型为torch.float32
    mask 是一个(1,H,W)，值位于0到1之间的张量，类型为torch.int64
    
    需要将image变成(H,W,3)的numpy数组，值位于0到255之间，类型为np.uint8
    需要将mask变成(H,W)的numpy数组，值位于0到255之间，类型为np.uint8
    """
    # 检查
    assert image.dim() == 4 and image.size(0) == 1 and image.size(1) == 3
    assert mask.dim() == 3 and mask.size(0) == 1
    assert image.dtype == torch.float32 and mask.dtype == torch.int64
    assert image.min() >= 0 and image.max() <= 1
    assert mask.min() >= 0 and mask.max() <= 1
    
    image = image.squeeze(0).permute(1,2,0).mul(255).byte().numpy()
    mask = mask.squeeze(0).mul(255).byte().numpy()
    return image, mask

模型的输出是(batch_size, 2, H, W)

需要对模型的输出进行如下处理：
1. 

In [ ]:
def crf_process(model, image):
    logit = model(image)
    
    logit = F.interpolate(logit, size=(H, W), mode="bilinear", align_corners=False)
    prob = F.softmax(logit, dim=1)[0].numpy()

    image = image.astype(np.uint8).transpose(1, 2, 0)
    prob = postprocessor(image, prob)
    label = np.argmax(prob, axis=0)